## William W. Johnsen Analysis A2

Starting off with importing packages, followed by setting print options for panda prints.

Finally reading the file I am working on.

In [1]:
# importing libraries
from sklearn.model_selection import GridSearchCV     # hyperparameter tuning
from sklearn.metrics import make_scorer              # customizable scorer                    # random number gen
import pandas            as pd                       # data science essentials
import matplotlib.pyplot as plt                      # data visualization
import seaborn           as sns                      # enhanced data viz
from sklearn.model_selection import train_test_split # train-test split
from sklearn.linear_model import LogisticRegression  # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

# reading file
original_df = pd.read_excel('Apprentice_Chef_Dataset.xlsx')

## Feature creation
In the cell below I am converting the 'FOLLOWED_RECOMMENDATIONS_PCT' column into actual numbers, instead of keeping percentages. If the percentage is 0, the number will remain 0, or else the number in that cell will end up being infinite. 

After that I am creating a couple of features:

- Average time each person use per click per visit. 
- Weekly orders.
- If someone has attended one or more Master Classes.
- If anybody does not have a last name, fill 'Unknown'.
- If anybody have same family name as first name, replace last name with 'Unkown'. 
- Make a dummy for 'Unkown' family name. 


In [2]:
original_df['RECOMMENDED_MEALS'] = 0 

for index, val in original_df.iterrows():
    if original_df.loc[index,'FOLLOWED_RECOMMENDATIONS_PCT'] > 0:
        original_df.loc[index, 'RECOMMENDED_MEALS'] = original_df.loc[index, 'TOTAL_MEALS_ORDERED']*(original_df.loc[index,'FOLLOWED_RECOMMENDATIONS_PCT']/100)
    elif original_df.loc[index,'FOLLOWED_RECOMMENDATIONS_PCT'] == 0:
         original_df.loc[index, 'RECOMMENDED_MEALS'] = 0
    else:
        print("what")
        

original_df['AVG_TIME_CLICK'] = original_df['AVG_TIME_PER_SITE_VISIT']/original_df['AVG_CLICKS_PER_VISIT']
# original_df['NOT_ON_SCHEDULE_PCT'] = ((original_df['EARLY_DELIVERIES']+original_df['LATE_DELIVERIES'])/original_df['TOTAL_MEALS_ORDERED'])*100
# original_df['CANCEL_PCT'] = ((original_df['CANCELLATIONS_BEFORE_NOON']+original_df['CANCELLATIONS_AFTER_NOON'])/original_df['TOTAL_MEALS_ORDERED'])*100
original_df['Weekly_Orders'] = original_df['TOTAL_MEALS_ORDERED']/52
# original_df['TOTAL_TIME'] = original_df['TOTAL_MEALS_ORDERED']*original_df['AVG_TIME_PER_SITE_VISIT']
# original_df['TOTAL_CLICKS'] = original_df['TOTAL_MEALS_ORDERED']*original_df['AVG_CLICKS_PER_VISIT']

original_df['MASTER_UP_1'] = 0
for index, val in original_df.iterrows():
    if original_df.loc[index, 'MASTER_CLASSES_ATTENDED'] >= 1:
        original_df.loc[index, 'MASTER_UP_1'] = 1 
        

        
# fill blank with unknown, assuming if the last and first is equal, they dont have a registered last name
original_df['FAMILY_NAME'] = original_df['FAMILY_NAME'].fillna('Unknown')


#if last name is equal first name, fill last with unknown
for index, val in original_df.iterrows():
    if original_df.loc[index, 'FIRST_NAME'] == original_df.loc[index, 'FAMILY_NAME']:
        original_df.loc[index, 'FAMILY_NAME'] = 'Unknown' 


        
original_df['UNKOWN'] = 0

for row, col in original_df.iterrows():
    if original_df.loc[row, 'FAMILY_NAME'] == 'Unknown':
        original_df.loc[row ,'UNKOWN'] = 1

### Lets hope the Targaryen are loyal people in real life, compared to the series. 

This is not part of the code anymore since they obviously are not. (Not significant for the model or prediction. 

Neither is the family with the most counts (The Freys). 

In [3]:
for index, val in original_df.iterrows():
    if original_df.loc[index,'FAMILY_NAME'].endswith('Targaryen'):
        original_df.loc[index, 'FAMILY_NAME'] = 'Targaryen'
    elif original_df.loc[index,'FAMILY_NAME'].endswith('Targaryen '):
        original_df.loc[index, 'FAMILY_NAME'] = 'Targaryen'

### Marking emails that contains a parantheses

The logic behind this is to mark if an email is actually getting to the client. If they have a special character like a parantheses, sending an email to them won't actually go through. 

In [4]:
# check if email has ( )  inside, this indicates that mail is invalid and will not reach client
original_df['INVALID_EMAIL'] = 0

for index, val in original_df.iterrows():
    if '(' in original_df.loc[index, 'EMAIL']:
        original_df.loc[index, 'INVALID_EMAIL'] = 1
    elif ')' in original_df.loc[index, 'EMAIL']:
        original_df.loc[index, 'INVALID_EMAIL'] = 1
    else: 
        original_df.loc[index, 'INVALID_EMAIL'] = 0

### One-hot-encoding the emails

Sorting the emails into:
- Professional email domains.
- Personal email domains.
- Junk email domains.

In [5]:
# Splitting emails into professional, personal and junk

# create a list to hold the different domains
p_lst = []


# looping over each email 
for i, col in original_df.iterrows():
    
    # splitting emails 
    split_m = original_df.loc[i, 'EMAIL'].split(sep = '@')
    
    # append to list
    p_lst.append(split_m)

    
# convert list to dataframe, followed by renaming the column names
email_df = pd.DataFrame(p_lst)

email_df.columns = ['name', 'domain']


# concatenating with original df
original_df = pd.concat([original_df, email_df.loc[:, 'domain']], axis = 1)


# defining the different group of domains.
professional_email = ['@mmm.com', '@amex.com', '@apple.com',  '@boeing.com','@caterpillar.com',
                      '@chevron.com','@cisco.com',  '@cocacola.com','@disney.com','@dupont.com',
                      '@exxon.com', '@ge.org','@goldmansacs.com','@homedepot.com','@ibm.com',
                      '@intel.com','@jnj.com','@jpmorgan.com','@mcdonalds.com','@merck.com',
                      '@microsoft.com','@nike.com','@pfizer.com','@pg.com', '@travelers.com',
                      '@unitedtech.com','@unitedhealth.com','@verizon.com', '@visa.com','@walmart.com']

personal_email     = ['@gmail.com', '@yahoo.com', '@protonmail.com']

junk_email         = ['@me.com', '@aol.com','@hotmail.com','@live.com','@msn.com','@passport.com']

# creating a list to hold the count of domains
p_lst              = []

for address in original_df['domain']:
    if '@' + address in professional_email:
        p_lst.append('professional')
        
    elif '@' + address in personal_email:
        p_lst.append('personal')
        
    elif '@' + address in junk_email:
        p_lst.append('junk')
    else:
        print('Unknown happening.')

# converting the list of domains into a new column in original_df
original_df['domain_group'] = pd.Series(p_lst)


# one hot encoding categorical data - here emails
one_hot_Email = pd.get_dummies(original_df['domain_group'])

# dropping categorical variables after they have been encoded
original_df = original_df.drop(['domain_group','domain'], axis = 1)

# joining codings together
original_df = original_df.join([one_hot_Email])

# saving new columns
new_columns = original_df.columns

### Counting names

Here I  count the number of names that a person has, but I didn't manage to do it in the manner I wanted to.

The idea was to calculate every name the person has, but if they had a title like "son of ______", everything inside parentheses should be counted as one. 

I did not manage to accomplish that.



In [6]:
#Number of names
original_df['N_NAMES'] = 0
for index, val in original_df.iterrows():
            original_df.loc[index, 'N_NAMES'] = len(original_df.loc[index, 'NAME'].split(sep = ' '))

### Dropping text variables

In [7]:
original_df = original_df.drop(labels= ['FAMILY_NAME','FOLLOWED_RECOMMENDATIONS_PCT','NAME', 'EMAIL', 'FIRST_NAME'],axis=1)

## Correlation check


In [8]:
# df_corr = original_df.corr().round(2)

# df_corr['CROSS_SELL_SUCCESS'].sort_values(ascending = False)

## Preparing data and target

In [9]:
original_data = original_df.drop('CROSS_SELL_SUCCESS', axis = 1)
original_target = original_df.loc[:, 'CROSS_SELL_SUCCESS']

## Scaling data

Scaling data because some observations is within thousands, some hundred, some tens and some are between 0 to 9. 

In [10]:
scaler = StandardScaler()

scaler.fit(original_data)

X_scaled = scaler.transform(original_data)

X_scaled_df = pd.DataFrame(X_scaled)

X_scaled_df.columns = original_data.columns

# train-test split
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled_df,
                                                        original_target,
                                                        test_size = 0.25,
                                                        random_state = 222)

## Initiating logistic model 

I am skipping the step of finding the proper variables with significant p-values. 


"In machine learning we usually don't look into these" 
(Ref. Professor Chase Kusterer)

A model might also find some information within unsignificant values. 

In [11]:
# INSTANTIATING a logistic regression model
logreg_scale = LogisticRegression(solver = 'lbfgs',
                            random_state = 222)


# FITTING the training data
logreg_scaled_fit = logreg_scale.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
logreg_scaled_pred = logreg_scaled_fit.predict(X_test_scaled)


# SCORING the results
log_reg_scaled_train = logreg_scaled_fit.score(X_train_scaled, y_train_scaled).round(4)
log_reg_scaled_test = logreg_scaled_fit.score(X_test_scaled, y_test_scaled).round(4)
log_reg_scaled_AUC   = roc_auc_score(y_true  = y_test_scaled, y_score = logreg_scaled_pred).round(4)

print('Training ACCURACY:', log_reg_scaled_train)
print('Testing  ACCURACY:', log_reg_scaled_test)
print('AUC Score        :', log_reg_scaled_AUC)

Training ACCURACY: 0.8012
Testing  ACCURACY: 0.7947
AUC Score        : 0.7422


In [12]:
# creating a confusion matrix
print(confusion_matrix(y_true = y_test_scaled,
                       y_pred = logreg_scaled_pred))

[[ 94  65]
 [ 35 293]]


### Hyperparameter Tuning

Running GridSearchCV on the LogisticRegression model.

In [13]:
# ########################################
# # GridSearchCV
# ########################################

# # declaring a hyperparameter space
# C_space          = pd.np.arange(0.1, 3.0, 0.1)
# warm_start_space = [True, False]


# # creating a hyperparameter grid
# param_grid = {'C'          : C_space,
#               'warm_start' : warm_start_space}


# # INSTANTIATING the model object without hyperparameters
# lr_tuned = LogisticRegression(solver = 'lbfgs',
#                               max_iter=1000,
#                               random_state = 222)


# # GridSearchCV object
# lr_tuned_cv = GridSearchCV(estimator  = lr_tuned,
#                            param_grid = param_grid,
#                            cv         = 3,
#                            scoring    = make_scorer(roc_auc_score,
#                                                     needs_threshold = False))


# # FITTING to the FULL DATASET (due to cross-validation)
# lr_tuned_cv.fit(X_scaled_df, original_target)


# # PREDICT step is not needed


# # printing the optimal parameters and best score
# print("Tuned Parameters  :", lr_tuned_cv.best_params_)
# print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

### Results of the GridSearchCV

Results are worse than the untuned Logistic model. 

In [14]:
# INSTANTIATING a logistic regression model
logreg = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=222, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=True)


# FITTING the training data
logreg_fit = logreg.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(X_test_scaled)


# SCORING the results
log_reg_scaled_train_tuned = logreg_fit.score(X_train_scaled, y_train_scaled).round(4)
log_reg_scaled_test_tuned  = logreg_fit.score(X_test_scaled, y_test_scaled).round(4)
log_reg_scaled_AUC_tuned   = roc_auc_score(y_true  = y_test_scaled, y_score = logreg_pred).round(4)

print('Training ACCURACY:', log_reg_scaled_train_tuned)
print('Testing  ACCURACY:', log_reg_scaled_test_tuned)
print('AUC Score        :', log_reg_scaled_AUC_tuned)

Training ACCURACY: 0.7985
Testing  ACCURACY: 0.7926
AUC Score        : 0.7342


In [15]:
# creating a confusion matrix
print(confusion_matrix(y_true = y_test_scaled,
                       y_pred = logreg_pred))

[[ 90  69]
 [ 32 296]]


## Model and trial dumping

This part is for models and feature engineering that turned out to be insignificant and/or bad. 

In [16]:
# NON SCALED TRAIN_TEST_SPLIT
X_train, X_test, y_train, y_test = train_test_split(
            original_data,
            original_target,
            test_size = 0.25,
            random_state = 222,
            stratify = original_target)

In [17]:
# Out_cancel_hi = 10
# Out_not_on_sc_hi = 40
# Out_avg_time_click_hi = 25
# Out_rec_meals_hi = 100
# Out_total_photos = 300
# Out_avg_prep_hi = 300
# Out_avg_time_visit_hi = 300
# Out_total_meals = 300
# Out_rev_hi = 4000


# original_df['Out_Cancel'] = 0

# condition_hi = original_df.loc[0:, 'Out_Cancel'][original_df['CANCEL_PCT'] > Out_cancel_hi] 


# original_df['Out_Cancel'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_Not_Schedule'] = 0

# condition_hi = original_df.loc[0:, 'Out_Not_Schedule'][original_df['NOT_ON_SCHEDULE_PCT'] > Out_not_on_sc_hi] 


# original_df['Out_Not_Schedule'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)


# original_df['Out_Avg_Time_Click'] = 0

# condition_hi = original_df.loc[0:, 'Out_Avg_Time_Click'][original_df['AVG_TIME_CLICK'] > Out_avg_time_click_hi] 


# original_df['Out_Avg_Time_Click'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_Rec_Meals'] = 0

# condition_hi = original_df.loc[0:, 'Out_Rec_Meals'][original_df['RECOMMENDED_MEALS'] > Out_rec_meals_hi] 


# original_df['Out_Rec_Meals'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)




# original_df['Out_TotPhoto'] = 0

# condition_hi = original_df.loc[0:, 'Out_TotPhoto'][original_df['TOTAL_PHOTOS_VIEWED'] > Out_total_photos] 


# original_df['Out_TotPhoto'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_Avg_prep'] = 0

# condition_hi = original_df.loc[0:, 'Out_Avg_prep'][original_df['AVG_PREP_VID_TIME'] > Out_avg_prep_hi] 


# original_df['Out_Avg_prep'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_Avg_Time_Visit'] = 0

# condition_hi = original_df.loc[0:, 'Out_Avg_Time_Visit'][original_df['AVG_TIME_PER_SITE_VISIT'] > Out_avg_time_visit_hi] 


# original_df['Out_Avg_Time_Visit'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_TotMeals'] = 0

# condition_hi = original_df.loc[0:, 'Out_TotMeals'][original_df['TOTAL_MEALS_ORDERED'] > Out_total_meals] 


# original_df['Out_TotMeals'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

# original_df['Out_Rev'] = 0

# condition_hi = original_df.loc[0:, 'Out_Rev'][original_df['REVENUE'] > Out_rev_hi] 


# original_df['Out_Rev'].replace(to_replace = condition_hi,
#                                 value   = 1,
#                                 inplace=True)

In [18]:
# INSTANTIATING the model object without hyperparameters
full_gbm_default = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.1,
                                              n_estimators  = 100,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 3,
                                              warm_start    = False,
                                              random_state  = 222)


# FIT step is needed as we are not using .best_estimator
full_gbm_default_fit = full_gbm_default.fit(X_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
full_gbm_default_pred = full_gbm_default_fit.predict(X_test_scaled)


# SCORING the results
full_scaled_gbm_train = full_gbm_default_fit.score(X_train_scaled, y_train_scaled).round(4)
full_scaled_gbm_test = full_gbm_default_fit.score(X_test_scaled, y_test_scaled).round(4)
full_scaled_gbm_AUC = roc_auc_score(y_true  = y_test_scaled,
                                          y_score = full_gbm_default_pred).round(4)

print('Training ACCURACY:', full_scaled_gbm_train)
print('Testing ACCURACY :', full_scaled_gbm_test)
print('AUC Score        :', full_scaled_gbm_AUC)

Training ACCURACY: 0.9013
Testing ACCURACY : 0.7906
AUC Score        : 0.736


In [19]:
# INSTANTIATING the model object without hyperparameters
full_gbm_non = GradientBoostingClassifier(loss          = 'deviance',
                                              learning_rate = 0.1,
                                              n_estimators  = 100,
                                              criterion     = 'friedman_mse',
                                              max_depth     = 3,
                                              warm_start    = False,
                                              random_state  = 222)


# FIT step is needed as we are not using .best_estimator
full_gbm_non_fit = full_gbm_non.fit(X_train, y_train)


# PREDICTING based on the testing set
full_gbm_non_pred = full_gbm_non_fit.predict(X_test)


# SCORING the results
full_non_gbm_train = full_gbm_default_fit.score(X_train, y_train_scaled).round(4)
full_non_gbm_test = full_gbm_default_fit.score(X_test, y_test_scaled).round(4)
full_non_gbm_AUC = roc_auc_score(y_true  = y_test,
                                          y_score = full_gbm_non_pred).round(4)

print('Training ACCURACY:', full_non_gbm_train)
print('Testing ACCURACY :', full_non_gbm_test)
print('AUC Score        :', full_non_gbm_AUC)

Training ACCURACY: 0.669
Testing ACCURACY : 0.6694
AUC Score        : 0.7458


In [20]:
# INSTANTIATING a random forest model with default values
rf_default = RandomForestClassifier(n_estimators     = 100,
                                    criterion        = 'gini',
                                    max_depth        = None,
                                    min_samples_leaf = 2,
                                    bootstrap        = True,
                                    warm_start       = False,
                                    random_state     = 222)

# FITTING the training data
rf_default_fit = rf_default.fit(X_train, y_train)


# PREDICTING based on the testing set
rf_default_fit_pred = rf_default_fit.predict(X_test)


# SCORING the results
rf_non_scaled_train = rf_default_fit.score(X_train, y_train).round(4)
rf_non_scaled_test = rf_default_fit.score(X_test, y_test).round(4)
rf_non_scaled_AUC = roc_auc_score(y_true  = y_test, y_score = rf_default_fit_pred).round(4)

print('Training ACCURACY:', rf_non_scaled_train)
print('Testing  ACCURACY:', rf_non_scaled_test)
print('AUC Score        :', rf_non_scaled_AUC)

Training ACCURACY: 0.9952
Testing  ACCURACY: 0.7474
AUC Score        : 0.6736


## Printing out the final results. 

The logistic model which was using scaled numbers, but was not tuned with GridSearchCV was the strongest model I could come up with.

To finalize this project I would consider this model a relatively weak one. I had a lot of struggles with creating features that would help me explain how the company could improve their chance of predicting which customers that would take advantage of the campaign "Halfway There". 

I am looking forward to the next class to get some insights of how the professor and other classmates have solved this assignment. 

In [21]:
# comparing results

print(f"""
KNN Model                  Train Score       Test Score       AUC 
----------------           ----------       ----------      ----------
Random Forest No Scale      {rf_non_scaled_train}             {rf_non_scaled_test}         {rf_non_scaled_AUC}
GBM non Scaled              {full_scaled_gbm_train}             {full_scaled_gbm_test}         {full_scaled_gbm_AUC}
GBM Scaled                  {full_non_gbm_train}              {full_non_gbm_test}         {full_non_gbm_AUC}
Logistic CV Scaled             {log_reg_scaled_train_tuned}             {log_reg_scaled_test_tuned}         {log_reg_scaled_AUC_tuned}
Logistic Scaled          {log_reg_scaled_train}             {log_reg_scaled_test}         {log_reg_scaled_AUC}



""")


KNN Model                  Train Score       Test Score       AUC 
----------------           ----------       ----------      ----------
Random Forest No Scale      0.9952             0.7474         0.6736
GBM non Scaled              0.9013             0.7906         0.736
GBM Scaled                  0.669              0.6694         0.7458
Logistic Scaled             0.7985             0.7926         0.7342
Logistic CV Scaled          0.8012             0.7947         0.7422




